In [1]:
%%bash
   export JAVA_OPTS="-Xmx8g"

In [2]:
!java -XX:+PrintFlagsFinal -version

[Global flags]
      int ActiveProcessorCount                     = -1                                        {product} {default}
    uintx AdaptiveSizeDecrementScaleFactor         = 4                                         {product} {default}
    uintx AdaptiveSizeMajorGCDecayTimeScale        = 10                                        {product} {default}
    uintx AdaptiveSizePolicyCollectionCostMargin   = 50                                        {product} {default}
    uintx AdaptiveSizePolicyInitializingSteps      = 20                                        {product} {default}
    uintx AdaptiveSizePolicyOutputInterval         = 0                                         {product} {default}
    uintx AdaptiveSizePolicyWeight                 = 10                                        {product} {default}
    uintx AdaptiveSizeThroughPutPolicy             = 0                                         {product} {default}
    uintx AdaptiveTimeWeight                       = 25          

In [3]:
!pip install kaggle
!kaggle datasets download -d sunnykakar/spotify-charts-all-audio-data

Mounted at /content/drive


In [ ]:
!unzip spotify-charts-all-audio-data.zip

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=8a1ffd180fbf7c4ac07ca9b0890e71ecad5dd9836345f395d09bf57ef3d4bc93
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Data Exploration

## Preliminary Exploration

In [7]:
# Inspect schema of an initial load of the data.

rawload = sqlContext.read.csv('merged_data.csv', header = True)
rawload.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- title: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- album: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- available_markets: string (nullable = true)
 |-- af_danceability: string (nullable = true)
 |-- af_energy: string (nullable = true)
 |-- af_key: string (nullable = true)
 |-- af_loudness: string (nullable = true)
 |-- af_mode: string (nullable = true)
 |-- af_speechiness: string (nullable = true)
 |-- af_acousticness: string (nullable = true)
 |-- af_instrumentalness: strin

In [8]:
# Show the first 5 rows of raw data
rawload.head(5)

[Row(_c0='0', title='Chantaje (feat. Maluma)', rank='1', date='2017-01-01', artist='Shakira', url='https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg', region='Argentina', chart='top200', trend='SAME_POSITION', streams='253019.0', track_id='6mICuAdrwEjh6Y6lroV2Kg', album='El Dorado', popularity='78.0', duration_ms='195840.0', explicit='False', release_date='2017-05-26', available_markets="['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM'

In [9]:
# Rename the first column, which refers to a primary key that was blandly named '_c0' by default.

df = rawload.withColumnRenamed('_c0', 'ID')

In [10]:
# Count observations in the dataset.

df_ct = df.count()
print(f'There are {df_ct} observations in this dataset.')

There are 26174269 observations in this dataset.


## Getting Summary Statistics for Numeric Attributes

In [11]:
# As the above schema shows, all attributes assume the string type, even though many of them are numeric. View a
# single observation to determine which attributes are numeric.

print(f'SAMPLE OBSERVATION:\n\n{df.take(1)}')

SAMPLE OBSERVATION:

[Row(ID='0', title='Chantaje (feat. Maluma)', rank='1', date='2017-01-01', artist='Shakira', url='https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg', region='Argentina', chart='top200', trend='SAME_POSITION', streams='253019.0', track_id='6mICuAdrwEjh6Y6lroV2Kg', album='El Dorado', popularity='78.0', duration_ms='195840.0', explicit='False', release_date='2017-05-26', available_markets="['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'T

In [12]:
# Evaluating the above observation allows us to establish which attributes are numeric.

numeric_attributes = ['ID', 'rank', 'streams', 'popularity', 'duration_ms', 'af_danceability', 'af_energy',
                      'af_key', 'af_loudness', 'af_mode', 'af_speechiness', 'af_acousticness',
                      'af_instrumentalness', 'af_liveness', 'af_valence', 'af_tempo',  'af_time_signature']
df[numeric_attributes].show(5)

+---+----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
| ID|rank| streams|popularity|duration_ms|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+---+----+--------+----------+-----------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
|  0|   1|253019.0|      78.0|   195840.0|          0.852|    0.773|   8.0|     -2.921|    0.0|        0.0776|          0.187|           3.05e-05|      0.159|     0.907| 102.034|              4.0|
|  1|   2|223988.0|      72.0|   259195.0|          0.663|     0.92|  11.0|      -4.07|    0.0|         0.226|        0.00431|           1.69e-05|      0.101|     0.533|  99.935|              4.0|
|  2|   3|21094

In [13]:
# We'll have to cast numeric attributes to floats to explore these attributes.

df = df.withColumns(dict(zip(numeric_attributes, [df[attribute].cast('float') for attribute in
                                                  numeric_attributes])))

In [14]:
# Confirm that type casting was correctly implemented.

print('UPDATED SCHEMA:\n')
df.printSchema()
print(f'\n\nSAMPLE OBSERVATION:\n\n{df.take(1)}')

UPDATED SCHEMA:

root
 |-- ID: float (nullable = true)
 |-- title: string (nullable = true)
 |-- rank: float (nullable = true)
 |-- date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: float (nullable = true)
 |-- track_id: string (nullable = true)
 |-- album: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- duration_ms: float (nullable = true)
 |-- explicit: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- available_markets: string (nullable = true)
 |-- af_danceability: float (nullable = true)
 |-- af_energy: float (nullable = true)
 |-- af_key: float (nullable = true)
 |-- af_loudness: float (nullable = true)
 |-- af_mode: float (nullable = true)
 |-- af_speechiness: float (nullable = true)
 |-- af_acousticness: float (nullable = true)
 |-- af_instrumentalness: f

In [15]:
# Now that we've casted numeric attributes to floats, we can generate summary statistics for them.

nas = df[numeric_attributes].describe()

In [ ]:
# We can display summary statistics in a visually appealing format by implementing the following.

for attribute in numeric_attributes:
    nas[['summary', attribute]].show()

+-------+-----------------+
|summary|               ID|
+-------+-----------------+
|  count|         26174269|
|   mean|      1.3087134E7|
| stddev|7555860.770833806|
|    min|              0.0|
|    max|      2.6174268E7|
+-------+-----------------+

+-------+------------------+
|summary|              rank|
+-------+------------------+
|  count|          26172127|
|   mean|  80.9233979339929|
| stddev|59.185732716119134|
|    min|               1.0|
|    max|             200.0|
+-------+------------------+

+-------+------------------+
|summary|           streams|
+-------+------------------+
|  count|          20318240|
|   mean|55266.480036705936|
| stddev|209601.75006362287|
|    min|            1001.0|
|    max|       1.9749704E7|
+-------+------------------+

+-------+-----------------+
|summary|       popularity|
+-------+-----------------+
|  count|         25853279|
|   mean|46.71695659958646|
| stddev|32.81488206233617|
|    min|              0.0|
|    max|          19176.0|

## Missing Data

In [ ]:
# Determine how many null values there are per attribute.

nulls_dict = {}
for attribute in df.columns:
    null_IDs = df[['ID']].where(df[attribute].isNull())
    nulls_dict[attribute] = (null_IDs, null_IDs.count())
    print(f'Attribute "{attribute}" contains {nulls_dict[attribute][1]} nulls. {(nulls_dict[attribute][1]/df_ct)*100}%')

In [ ]:
#Exploring categorical data

categorical_columns = ['region', 'chart', 'trend', 'explicit']
for column in categorical_columns:
    df.groupBy(column).count().orderBy('count', ascending=False).show()

In [ ]:
#The outputs above can give us a hint on what we can remove from dataset

## Data Visualization

### Popularity vs Streams

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import col

# Select subset of data
data_subset = df.select("popularity", "streams").dropna().sample(fraction=0.2)

pandas_df = data_subset.toPandas()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(pandas_df['popularity'], pandas_df['streams'], alpha=0.5)
plt.title('Popularity vs Streams')
plt.xlabel('Popularity')
plt.ylabel('Streams')
plt.grid(True)
plt.show()

###Duration vs Popularity

In [ ]:
data_subset = df.select(
    col("duration_ms"),
    col("popularity")
).na.drop()

sampled_data = data_subset.sample(fraction=0.1)
pandas_df = sampled_data.toPandas()

plt.figure(figsize=(10, 6))
plt.scatter(pandas_df['duration_ms'], pandas_df['popularity'], alpha=0.5, color='green')
plt.title('Track Duration vs Popularity')
plt.xlabel('Duration in milliseconds')
plt.ylabel('Popularity')
plt.grid(True)
plt.show()

### Stream vs Explicitness

In [ ]:
explicit_streams = df.select(
    col("explicit"),
    col("streams")
).na.drop()

sampled_explicit_streams = explicit_streams.sample(fraction=0.1)

# Collect and plot
pandas_explicit_streams = sampled_explicit_streams.toPandas()

plt.figure(figsize=(10, 6))
plt.scatter(pandas_explicit_streams['explicit'], pandas_explicit_streams['streams'], alpha=0.5, color='red')
plt.title('Streams vs Explicitness')
plt.xlabel('Explicit')
plt.ylabel('Streams')
plt.grid(True)
plt.show()

###Popularity vs Region

In [ ]:
region_popularity = df.groupBy("region").agg(avg("popularity").alias("avg_popularity")).na.drop()

sampled_region_popularity = region_popularity.sample(fraction=0.4)

pandas_region_popularity = sampled_region_popularity.toPandas()

plt.figure(figsize=(12, 8))
pandas_region_popularity.plot(kind='bar', x='region', y='avg_popularity', color='purple')
plt.title('Average Popularity by Region')
plt.xlabel('Region')
plt.ylabel('Average Popularity')
plt.xticks(rotation=45)
plt.show()

### Energy vs Loudness

In [ ]:
energy_loudness = df.select(
    col("af_energy"),
    col("af_loudness")
).na.drop().sample(fraction=0.1)

pandas_energy_loudness = energy_loudness.toPandas()

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(pandas_energy_loudness['af_energy'], pandas_energy_loudness['af_loudness'], alpha=0.5)
plt.title('Energy vs Loudness of Tracks')
plt.xlabel('Energy')
plt.ylabel('Loudness (dB)')
plt.grid(True)
plt.show()

###Tempo vs Danceability

In [ ]:
tempo_danceability = df.select(
    col("af_tempo"),
    col("af_danceability")
).na.drop().sample(fraction=0.1)

pandas_tempo_danceability = tempo_danceability.toPandas()

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(pandas_tempo_danceability['af_tempo'], pandas_tempo_danceability['af_danceability'], alpha=0.5, color='blue')
plt.title('Tempo vs Danceability')
plt.xlabel('Tempo (Beats per Minute)')
plt.ylabel('Danceability')
plt.grid(True)
plt.show()

###Valence vs Acousticness

In [ ]:
valence_acousticness = df.select(
    col("af_valence"),
    col("af_acousticness")
).na.drop().sample(fraction=0.1)

pandas_valence_acousticness = valence_acousticness.toPandas()

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(pandas_valence_acousticness['af_valence'], pandas_valence_acousticness['af_acousticness'], alpha=0.5, color='magenta')
plt.title('Valence vs Acousticness')
plt.xlabel('Valence')
plt.ylabel('Acousticness')
plt.grid(True)
plt.show()

### Popularity by Release Year

In [ ]:
from pyspark.sql.functions import year

popularity_year = df.select(
    year(col("release_date")).alias("release_year"),
    col("popularity")
).na.drop().sample(fraction=0.3)

pandas_popularity_year = popularity_year.toPandas()

# Plotting
plt.figure(figsize=(12, 8))
plt.scatter(pandas_popularity_year['release_year'], pandas_popularity_year['popularity'], alpha=0.5, color='orange')
plt.title('Popularity by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Popularity')
plt.grid(True)
plt.show()